In [2]:
import itertools
#import collections

In [3]:
N = ToricLattice(7)
K = N([-3, 1, 1, 1, 1, 1, 1]) # canonical divisor
E = identity_matrix(QQ, 7)[:,1:].columns() # E is the set of exceptional curves, so the basis is L, E[0], .. , E[5]
E = [N(e) for e in E]
Line = lambda exceptional_curves : N([i/3 for i in (-K + sum(exceptional_curves)) ])
L = Line(E)

Q = diagonal_matrix([1]+6*[-1])
dot = lambda a, b: a*Q*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])

minus_one_curves = E + [L-ei-ej for ei,ej in itertools.combinations(E, 2)] + [-K-L+e for e in E]
curves_disjoint_with = lambda curves: [c for c in minus_one_curves if all(dot(c,c2)==0 for c2 in curves)]

double_intersections = [(a,b) for a,b in itertools.combinations(minus_one_curves, 2) if dot(a,b)==1]
triple_intersections = [(a,b,c) for a,b,c in itertools.combinations(minus_one_curves, 3) if dot(a,b)+dot(b,c)+dot(a,c)==3]

NE = Cone(minus_one_curves)
Ample = Cone([N(Q*ray) for ray in NE.dual().rays()])

# take one Fujita face for each isomorphism class (determined by CPW, may check explicitly later)
# and turn them into subcones in Mori cone by adding -K

NE_face_types_rays = {
        "B6": E,
        "B5": E[:5],
        "B5'": E[:4]+[L-E[4]-E[5]], # B(5) for P1xP1. Another type of independent set of 5 (-1)-curves, this one is maximal
        "B4": E[:4],
        "B3": E[:3],
        "B2": E[:2],
        "B1": E[:1],
        #'C6': E[:5]+[L-E[5]-e for e in E[:5]], # C(6). Schlafli graph is 4-ultrahomogeneous, and, after fixing two pairs, three others are defined uniquely
        "C5": E[:5]+[L-E[5]], # здесь и ниже - части C6, из подобных которым складывается весь С6 
        "C5'": E[:4]+[L-E[5]-E[4], L-E[5]],
        "C4": E[:4]+[L-E[5]],
        "C3": E[:3]+[L-E[5]],
        "C2": E[:2]+[L-E[5]],
        "C1": E[:1]+[L-E[5]],
        "C0": E[:0]+[L-E[5]],
    }
cone_of_type = {label:Cone(rays+[-K]).intersection(Ample) for label,rays in NE_face_types_rays.items()}

# returns True if rel.int.(cone1) is in rel.int.(cone2)
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

In [4]:
NE_face_types_rays["C0"]

[N(1, 0, 0, 0, 0, 0, -1)]

In [3]:
# Constructing cylinders.

def independent_sets(vectors, size = 6):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]

#cylinder1 = lambda E : E + [Line(E)-E[-1], -K-Line(E)+E[-1]]   
def cylinder_generator_P2(E):
    '''
    E is a list of exceptional curves, and e is one of them
    returns 
    1. list of rays of the cone of divisors H such that the cylinder (P^2 minus conic through E-e and tangent line through e) is H-polar
    2. list of (-1)-curves lying in the complement of the cylinder (or a union of cylinders if many)
    '''
    for e in E:
        tangent = Line(E) - e
        conic = - K - tangent
        curves = E+[conic]
        yield curves + [tangent]

def cylinder_generator_1(E):
    """
    cylinder complement to E and lines through blowdowns of e and one of E-e
    """
    for e in E:
        curves = E+[Line(E)-e-f for f in E if f!=e]
        yield curves

def cylinder_generator_P1(E):
    '''
    CPW Example 4.1.6 and Lemma 4.2.2 for contraction of E1..E4, L-E5-E6.
    '''
    for i,j in itertools.permutations(range(len(E)),int(2)):
        complement = [k for k in range(len(E)) if (k!=i) and (k!=j)]
        conic = -K-Line(E)+E[i]
        curves = [E[k] for k in complement]+[Line(E)-E[i]-E[j], conic]
        yield curves + [Line(E)-E[i],Line(E)-E[j]]

def cylinders_of_type(cylinder_generator_list):
    '''
    returns a list of cones of cylinders of considered type
    '''
    for cylinder_generator in cylinder_generator_list:
        for blowdown in independent_sets(minus_one_curves):
            yield from cylinder_generator(blowdown)

# polarity cone of a collection of cylinders
def Pol(collection):
    result = Cone([],lattice=N.dual()).dual() # ambient space
    for cylinder in collection:
        result = result.intersection(Cone(cylinder['rays']))
    return result

def minus_one_curves_in_complement(collection):
    return [curve for curve in minus_one_curves if all(curve in cylinder['rays'] for cylinder in collection)]
    
# forbidden cone of a collection of cylinders
def Forb(collection):
    return Cone(minus_one_curves_in_complement(collection),lattice=N)

cylinders = [{
    'rays': cylinder_rays,
    'index': i,
} for i, cylinder_rays in enumerate(cylinders_of_type([cylinder_generator_P2,cylinder_generator_P1]))]

for u in cylinders:
    u['Pol'] = Pol([u])
    u['Forb'] = Forb([u])


In [139]:

def is_polar_wrt(cylinder, cones):
    return all(is_covered_by(cone,cylinder['Pol']) for cone in cones)

intersection_point_in_cylinder = lambda intersection, cylinder: all(curve not in cylinder['rays'] for curve in intersection)
intersection_point_in_cylinders = lambda intersection, cylinders: any(intersection_point_in_cylinder(intersection, c) for c in cylinders)
def complement_points(cylinders):
    return {
        'double': [intersection for intersection in double_intersections if not intersection_point_in_cylinders(intersection, cylinders)],
        'triple': [intersection for intersection in triple_intersections if not intersection_point_in_cylinders(intersection, cylinders)],
    }

In [133]:
point = double_intersections[0]
cylinder = collection_of_type['B2'][0]
point in complement_points([cylinder])['double']

True

In [155]:

def point_eliminating_cylinder(pool, U, seed=123456):
    polarity = Pol(U).intersection(Ample) if len(U)>0 else Ample
    double_points = complement_points(U)['double']
    print('double points in complement: ', len(double_points))
    polarity_agrees = lambda cone: is_covered_by(cone, Ample) and cone.codim()==0
    candidates = [c for c in pool if polarity_agrees(c['Pol'].intersection(polarity))]
    print('candidates: ', len(candidates))
    if len(candidates) == 0:
        return None
    candidates_with_counts = [(c, len([point for point in double_points if intersection_point_in_cylinder(point, c)])) for c in candidates]
    max_count = max([c[1] for c in candidates_with_counts])
    print('points eliminated: ', max_count)
    if max_count == 0:
        return None
    best_candidates = [candidate for candidate, count in candidates_with_counts if count == max_count]
    print('best candidates: ', len(best_candidates))
    print('returning ', best_candidates[0])
    return best_candidates[0] #[seed%len(best_candidates)]

def eliminate_points(pool=cylinders, collection=[]):
    new_collection = collection[:]
    while True:
        new_cylinder = point_eliminating_cylinder(pool, new_collection)
        #print('new cylinder', new_cylinder)
        if new_cylinder == None:
            break
        new_collection.append(new_cylinder)
        print('New collection length:\n', len(new_collection))
    return new_collection
    



#total_collection = eliminate_points(cylinders[200:300], U)

In [107]:
#complement_points(collection_of_type['C2'])
p = Pol(collection_of_type['B2'])
is_covered_by(cone_of_type['C2'], p)

False

In [154]:
#total_collection = U+cylinders[204]
#for key, cone in cone_of_type.items():
#    print(key, Pol(total_collection).intersection(cone))

def eliminate_cylinders(collection):
    if len(complement_points(collection)['double'])!=0:
        print('this collection does not cover all double points')
        return collection
    print('deleting cylinders, remaining:')
    while True:
        for i in range(len(collection)):
            collection_without_cylinder = collection[:i] + collection[i+1:]
            if len(complement_points(collection_without_cylinder)['double'])==0:
                del collection[i]
                print(len(collection), end=' ')
                break
        else:
            print('no cylinders to delete')
            break
    return collection
            

#Pol(total_collection).intersection(cone_of_type['C0']).rays()
#print(complement_points(total_collection)['double'])
#another_collection = total_collection[:]
#shorter_collection = eliminate_cylinders(another_collection)
#print(len(another_collection))
#len(shorter_collection)

In [152]:
cone_types_list_1 = [cone_of_type[i] for i in ["B6","B5","B4","B3","B2","B1",]]

collection_of_type=dict()
for i, cone in cone_of_type.items():
    collection_of_type[i]=[c for c in cylinders if is_polar_wrt(c,[cone])]
    collection_of_type[i] = eliminate_cylinders(collection_of_type[i])

for i, collection in collection_of_type.items():
    print(i,": ", len(complement_points(collection)['double']) ," points ", len(collection) ," cylinders")
#u_1=cylinders[5]
#Forb(U).intersection(Ample)

this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
this collection does not cover all double points
B6 :  60  points  6  cylinders
B5 :  60  points  6  cylinders
B5' :  50  points  10  cylinders
B4 :  60  points  4  cylinders
B3 :  36  points  6  cylinders
B2 :  20  points  12  cylinders
B1 :  10  points  16  cylinders
C5 :  65  points  1  cylinders
C5' :  50  points  10  cylinders
C4 :  55  points  5  cylinder

In [156]:
collection_c0 = eliminate_points(collection=collection_of_type['C0'])
shorter_collection_c0 = eliminate_cylinders(collection_c0)
Pol(shorter_collection_c0)

double points in complement:  5
candidates:  656
points eliminated:  4
best candidates:  400
returning  {'rays': [N(1, 0, 0, -1, 0, -1, 0), N(1, 0, 0, -1, -1, 0, 0), N(0, 0, 0, 0, 0, 0, 1), N(0, 1, 0, 0, 0, 0, 0), N(1, 0, -1, -1, 0, 0, 0), N(1, -1, 0, 0, 0, 0, -1), N(2, 0, -1, -1, -1, -1, 0), N(1, 0, 0, -1, 0, 0, 0)], 'index': 572, 'Pol': 7-d cone in 7-d lattice N, 'Forb': 6-d cone in 7-d lattice N}
New collection length:
 81
double points in complement:  1
candidates:  339
points eliminated:  1
best candidates:  184
returning  {'rays': [N(1, 0, 0, 0, -1, -1, 0), N(1, -1, 0, 0, 0, -1, 0), N(1, -1, 0, 0, -1, 0, 0), N(0, 0, 0, 0, 0, 0, 1), N(0, 0, 0, 1, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(1, 0, 0, -1, 0, 0, -1), N(2, -1, -1, 0, -1, -1, 0)], 'index': 113, 'Pol': 7-d cone in 7-d lattice N, 'Forb': 7-d cone in 7-d lattice N}
New collection length:
 82
double points in complement:  0
candidates:  238
points eliminated:  0
deleting cylinders, remaining:
81 80 79 78 77 76 75 74 73 72 71 70 69 

7-d cone in 7-d lattice N

In [173]:
shorter_collection_c0
for c in shorter_collection_c0:
    print('\n',c['index'])
    for i, ray in enumerate(c['rays']):
        print(i+1,ray)



 1787
1 N(1, 0, 0, -1, 0, 0, -1)
2 N(1, 0, -1, 0, 0, 0, -1)
3 N(1, -1, 0, 0, 0, 0, -1)
4 N(0, 0, 0, 0, 0, 1, 0)
5 N(0, 0, 0, 0, 1, 0, 0)
6 N(1, 0, 0, 0, -1, -1, 0)
7 N(2, -1, -1, -1, 0, 0, -1)
8 N(1, 0, 0, 0, 0, 0, -1)

 1792
1 N(1, 0, 0, 0, -1, 0, -1)
2 N(1, 0, -1, 0, 0, 0, -1)
3 N(1, -1, 0, 0, 0, 0, -1)
4 N(0, 0, 0, 0, 0, 1, 0)
5 N(0, 0, 0, 1, 0, 0, 0)
6 N(1, 0, 0, -1, 0, -1, 0)
7 N(2, -1, -1, 0, -1, 0, -1)
8 N(1, 0, 0, 0, 0, 0, -1)

 1797
1 N(1, 0, 0, 0, -1, 0, -1)
2 N(1, 0, 0, -1, 0, 0, -1)
3 N(1, -1, 0, 0, 0, 0, -1)
4 N(0, 0, 0, 0, 0, 1, 0)
5 N(0, 0, 1, 0, 0, 0, 0)
6 N(1, 0, -1, 0, 0, -1, 0)
7 N(2, -1, 0, -1, -1, 0, -1)
8 N(1, 0, 0, 0, 0, 0, -1)

 1802
1 N(1, 0, 0, 0, -1, 0, -1)
2 N(1, 0, 0, -1, 0, 0, -1)
3 N(1, 0, -1, 0, 0, 0, -1)
4 N(0, 0, 0, 0, 0, 1, 0)
5 N(0, 1, 0, 0, 0, 0, 0)
6 N(1, -1, 0, 0, 0, -1, 0)
7 N(2, 0, -1, -1, -1, 0, -1)
8 N(1, 0, 0, 0, 0, 0, -1)

 572
1 N(1, 0, 0, -1, 0, -1, 0)
2 N(1, 0, 0, -1, -1, 0, 0)
3 N(0, 0, 0, 0, 0, 0, 1)
4 N(0, 1, 0, 0, 0, 0, 0)
5 N(1, 0, 

In [194]:
#p = Pol(shorter_collection_c0).intersection(Ample)
#rays = p.rays()
p.relative_interior_contains(sum([rays[i] for i in [1,2,4,5,6]]))
sum([rays[i] for i in [1,2,4,5,6]])

N(18, -3, -1, -7, -3, -3, -10)

In [196]:
Pol(shorter_collection_c0).rays()

N(1,  0,  0,  0,  0,  0, -1),
N(3, -1,  0, -1, -1,  0, -2),
N(3, -1,  0, -1,  0, -1, -2),
N(2,  0,  0, -1,  0,  0, -1),
N(5,  0,  0, -2, -1, -1, -3),
N(6, -1, -1, -3, -1, -1, -3),
N(1,  0,  0,  0,  0,  0,  0),
N(7, -1,  0, -3, -1, -2, -4),
N(3, -1, -1, -1, -1, -1, -1)
in 7-d lattice N

In [109]:
for i, collection in collection_of_type.items():
    print(i,": ", len(complement_points(collection)['double']) ," points ", len(collection) ," cylinders")

print([c['index'] for c in collection_of_type['B2']])
#print([c['index'] for c in collection_of_type['C2']])
for c in collection_of_type['B2']:
    print('\n')
    for ray in c['rays']:
        print(ray)
complement_points(collection_of_type['B2'])


B6 :  45  points  6  cylinders
B5 :  45  points  6  cylinders
B5' :  75  points  10  cylinders
B4 :  55  points  4  cylinders
B3 :  27  points  6  cylinders
B2 :  0  points  6  cylinders
B1 :  5  points  16  cylinders
C5 :  70  points  1  cylinders
C5' :  75  points  10  cylinders
C4 :  80  points  5  cylinders
C3 :  57  points  10  cylinders
C2 :  0  points  6  cylinders
C1 :  1  points  48  cylinders
C0 :  0  points  12  cylinders
[11, 17, 23, 29, 71, 137]


N(1, 0, 0, 0, 0, -1, -1)
N(1, 0, 0, 0, -1, 0, -1)
N(1, 0, 0, 0, -1, -1, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, -1, -1, 0, 0, 0)
N(2, -1, 0, 0, -1, -1, -1)


N(1, 0, 0, 0, 0, -1, -1)
N(1, 0, 0, -1, 0, 0, -1)
N(1, 0, 0, -1, 0, -1, 0)
N(0, 0, 0, 0, 1, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, -1, 0, -1, 0, 0)
N(2, -1, 0, -1, 0, -1, -1)


N(1, 0, 0, 0, -1, 0, -1)
N(1, 0, 0, -1, 0, 0, -1)
N(1, 0, 0, -1, -1, 0, 0)
N(0, 0, 0, 0, 0, 1, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0

[-1  0  0  0  0  0  1  0]
[ 0 -1  0  0  0  0  1  0]
[ 0  0 -1  0  0  0  1  0]
[ 0  0  0 -1  0  0  1  0]
[ 0  0  0  0 -1  0  1  0]
[ 0  0  0  0  0 -1  0  1]
[ 1  1  1  1  1  0 -1  2]
[ 0  0  0  0  0  1  2  0]

In [117]:

gram_matrix(collection_of_type['B2'][0]['rays'])

[-1  0  0  0  0  0  1  0]
[ 0 -1  0  0  0  0  1  0]
[ 0  0 -1  0  0  0  1  0]
[ 0  0  0 -1  0  0  1  0]
[ 0  0  0  0 -1  0  1  0]
[ 0  0  0  0  0 -1  0  1]
[ 1  1  1  1  1  0 -1  2]
[ 0  0  0  0  0  1  2  0]

In [152]:
cone_types_list_2 = [cone_of_type[i] for i in ["C5",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_2)]
[u['index'] for u in U]


[0]

In [153]:
cone_types_list_7 = [cone_of_type[i] for i in ["C3","C2","C1","C0",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_7)]
[u['index'] for u in U]


[437, 442, 463, 468, 498, 528, 588, 618, 918, 948]

In [102]:
cone_types_list_3 = [cone_of_type[i] for i in ["B5'","C5'",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_3)]
[u['index'] for u in U]
#Forb([cylinders[437],cylinders[432]]).rays()


[432, 437, 468, 473, 498, 503, 588, 593, 918, 923]

In [122]:
cone_types_list_4 = [cone_of_type[i] for i in ["C4",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_4)]
[u['index'] for u in U]
#U4 = [cylinders[437]]
#Forb(U4).rays()


[437, 468, 498, 588, 918]

In [119]:
cone_of_type["C4'"] = cylinders[437]['Forb'].intersection(cone_of_type['C4'])
U=[c for c in cylinders if is_polar_wrt(c,[cone_of_type["C4'"]])]
[u['index'] for u in U]
U5=[c for c in cylinders if c['index'] in [437,442]]
Forb(U5).intersection(Ample)

0-d cone in 7-d lattice N

In [183]:
cone_types_list_5 = [cone_of_type[i] for i in ["C4'","C3","C2","C1","C0",]]
print(Forb(U3).rays())
print(NE_face_types_rays["C3"])


N(0,  1,  0,  0,  0,  0,  0),
N(0,  0,  1,  0,  0,  0,  0),
N(0,  0,  0,  1,  0,  0,  0),
N(0,  0,  0,  0,  1,  0,  0),
N(1,  0,  0,  0,  0, -1, -1),
N(2, -1, -1, -1, -1,  0, -1)
in 7-d lattice N
[N(0, 1, 0, 0, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(1, 0, 0, 0, 0, 0, -1)]


In [128]:
for i in  [432,437,442]:
    print(i)
    for ray in cylinders[i]['rays']:
        print(ray)

    

432
N(0, 0, 0, 0, 1, 0, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, 0, -1, -1)
N(2, -1, -1, -1, -1, -1, 0)
N(1, 0, 0, 0, 0, 0, -1)
N(1, 0, 0, 0, 0, -1, 0)
437
N(0, 0, 0, 0, 1, 0, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, 0, -1, -1)
N(2, -1, -1, -1, -1, 0, -1)
N(1, 0, 0, 0, 0, -1, 0)
N(1, 0, 0, 0, 0, 0, -1)
442
N(0, 0, 0, 0, 0, 1, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, -1, 0, -1)
N(2, -1, -1, -1, 0, -1, -1)
N(1, 0, 0, 0, -1, 0, 0)
N(1, 0, 0, 0, 0, 0, -1)


In [148]:
U1=[c for c in cylinders if c['index'] in [5]]
U2=[c for c in cylinders if c['index'] in [0]]
U3=[c for c in cylinders if c['index'] in [437]]
U4=[c for c in cylinders if c['index'] in [437,432]]
U5=[c for c in cylinders if c['index'] in [437,442]]
for key, cone in cone_of_type.items():
    for i,U in enumerate([U1,U2,U3,U4,U5]):
        #print(i,key,is_covered_by(cone,Pol(U)))
        print(i,Pol(U))

0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d lattice N
1 7-d cone in 7-d lattice N
2 7-d cone in 7-d lattice N
3 7-d cone in 7-d lattice N
4 7-d cone in 7-d lattice N
0 7-d cone in 7-d la

In [159]:
is_covered_by(cone_of_type["B5'"],Pol(U3))
Pol(U3).contains(E[4])

False

In [186]:
G=Forb(U3).intersection(Cone(NE_face_types_rays["C5'"]+[-K]))
G.dual().lines()

M(1, 0, 0, 0, 0, -1, 2)
in 7-d lattice M

In [166]:
for ray in NE_face_types_rays["C5'"]+[-K]:
    if ray not in Pol(U3):
        print(ray)

In [191]:
NE_face_types_rays["B5'"]
dot(L+E[4]-2*E[5],2*(L-E[5])-K)

0

In [207]:
curves = E[:4]+[L-E[4]-E[5]]
q = minus_one_curves[25]
[e for e in minus_one_curves if all(dot(e,c)==1 for c in curves)]


[N(2, -1, -1, -1, -1, 0, -1), N(2, -1, -1, -1, -1, -1, 0)]

In [209]:
for i,j in double_intersections:
    if i+j==L-E[4]:
        print(i,j)

N(0, 1, 0, 0, 0, 0, 0) N(1, -1, 0, 0, 0, -1, 0)
N(0, 0, 1, 0, 0, 0, 0) N(1, 0, -1, 0, 0, -1, 0)
N(0, 0, 0, 1, 0, 0, 0) N(1, 0, 0, -1, 0, -1, 0)
N(0, 0, 0, 0, 1, 0, 0) N(1, 0, 0, 0, -1, -1, 0)
N(0, 0, 0, 0, 0, 0, 1) N(1, 0, 0, 0, 0, -1, -1)
